In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate, learning_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, classification_report

%matplotlib inline

In [1]:
!pip install pandas numpy seaborn sklearn matplotlib

In [ ]:
df = pd.read_csv('data/churn.csv', index_col=0)
df.head()

In [ ]:
labels = df.churn
features = df.drop('churn', axis=1)
features.head(20)

In [ ]:
drop_columns = ['host_response_time', 
                'calendar_updated',
                'days_since_last_review']

In [ ]:
features.drop(drop_columns, axis=1).info()

In [ ]:
pd.get_dummies(features.drop(drop_columns, axis=1))

In [ ]:
feature_matrix = pd.get_dummies(features.drop(drop_columns, axis=1)).values
feature_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, labels, test_size=0.3)

In [ ]:
print(len(X_train))
print(len(X_test))

In [ ]:
# create model (estimator) object
model = LogisticRegression(solver='liblinear')
model

In [ ]:
model.get_params()

In [ ]:
# fit model to training data
model.fit(X_train, y_train)

In [ ]:
model.coef_

In [ ]:
# make predictions
predictions = model.predict(X_test)
predictions

In [ ]:
model.predict_proba(X_test)

In [ ]:
# evaluate model
accuracy = (predictions == y_test).sum() / len(y_test)
accuracy

In [ ]:
model.score(X_test, y_test)

In [ ]:
df.churn.value_counts()

## Class Imbalance

In [ ]:
down_sample = df[df.churn == False].sample(32575)
sub_sample = pd.concat([down_sample, df[df.churn == True].sample(32575)])

In [ ]:
sub_sample.churn.value_counts()

In [ ]:
labels = sub_sample.churn
drop_columns = ['host_response_time', 
                'calendar_updated',
                'days_since_last_review', 'churn']
features = sub_sample.drop(drop_columns, axis=1)
feature_matrix = pd.get_dummies(features).values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, labels, test_size=0.3)

In [ ]:
# create model (estimator) object
clf = LogisticRegression(penalty='l1', solver='liblinear')

# fit model to training data
clf.fit(X_train, y_train)

# make predictions
predictions = clf.predict(X_test)

# evaluate model
accuracy = (predictions == y_test).sum() / len(y_test)
print(accuracy)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
print(f"Training took {clf.n_iter_[0]} iterations, since this is less than {clf.max_iter} it has converged")

## k-fold Cross Validation

In [ ]:
scores = cross_val_score(clf, X_train, y_train, cv=5)

In [ ]:
cross_val_score??

In [ ]:
scores

In [ ]:
scores.mean()

In [ ]:
scores.var()

In [ ]:
np.array([0.2, .2, .9, .8, .9, .5, .8, .7, .7, .7]).mean()

In [ ]:
np.array([0.2, .2, .9, .8, .9, .5, .8, .7, .7, .7]).var()

In [ ]:
clf.score(X_test, y_test)

In [ ]:
model = KNeighborsClassifier()
clf = model.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
clf = model.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
clf.feature_importances_

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
model = RandomForestClassifier()
clf = model.fit(X_train, y_train)
clf.score(X_test, y_test)

y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure(figsize=(5, 5))
plot_confusion_matrix(cm)

In [ ]:
model = KNeighborsClassifier()
clf = model.fit(X_train, y_train)
clf.score(X_test, y_test)

y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure(figsize=(5, 5))
plot_confusion_matrix(cm)

In [ ]:
y_pred

In [ ]:
y_score

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
model = KNeighborsClassifier()
clf = model.fit(X_train, y_train)
clf.score(X_test, y_test)

y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)


fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC plot for Logistic Regression for Airbnb Churn')
plt.legend(loc="lower right")
plt.show()

In [ ]:
model = RandomForestClassifier()
clf = model.fit(X_train, y_train)
clf.score(X_test, y_test)

y_pred = clf.predict(X_test)
y_score = clf.predict_proba(X_test)


fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC plot for Logistic Regression for Airbnb Churn')
plt.legend(loc="lower right")
plt.show()

In [ ]:
def plot_roc(y_test, y_score):
    fpr, tpr, thresholds = roc_curve(y_test, y_score[:, 1])

    plt.figure(figsize=(10,10))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc(fpr, tpr))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC plot for Logistic Regression for Airbnb Churn')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
scores = cross_validate(clf, X_train, y_train, return_train_score=True, cv=5)

In [ ]:
scores